In [3]:
import warnings
warnings.filterwarnings('ignore')

# %%script python -- arg1 arg2 arg3

import ivy
ivy.set_backend("torch")
from googlenet import inceptionNet_v1
from ivy_models_tests import helpers

def test():
    model = inceptionNet_v1(pretrained=False)
    output = model(torch.randn(1, 3, 224, 224))
    print(output.size())

test()

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 224, 3, 224] to have 3 channels, but got 224 channels instead

# Start

In [4]:
# global
import builtins
import ivy
import ivy_models
import numpy as np
import torch

In [ ]:
model = inceptionNet_v1(pretrained=False)
output = model(torch.randn(1, 3, 224, 224))
print(output.size())

In [ ]:
def inceptionNet_v1(pretrained=True):
    """InceptionNet-V1 model"""
    if not pretrained:
        return GoogLeNet()

    reference_model = GoogLeNet()
    url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
    w_clean = ivy_models.helpers.load_torch_weights(
        url,
        reference_model,
        raw_keys_to_prune=["num_batches_tracked"],
        custom_mapping=_inceptionNet_torch_weights_mapping,
    )
    return GoogLeNet(v=w_clean)


In [12]:
def _inceptionNet_torch_weights_mapping(old_key, new_key):
    W_KEY = ["conv1/weight", "conv2/weight", "downsample/0/weight"]
    new_mapping = new_key
    if builtins.any([kc in old_key for kc in W_KEY]):
        new_mapping = {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
    return new_mapping


In [5]:
class GoogLeNet(ivy.Module):
    def __init__(self, num_classes=1000, v=None):
        super(GoogLeNet, self).__init__()
        
        if v is not None:
            self.v = v
        
        self.conv1 = ConvBlock(
            3,
            64,
            kernel_size=[
                7,
            ],
            stride=2,
            padding=3,
        )
        self.pool1 = ivy.MaxPool2D(3, 2, 0)
        self.conv2 = ConvBlock(
            64,
            64,
            kernel_size=[
                1,
            ],
            stride=1,
            padding=0,
        )
        self.conv3 = ConvBlock(
            64,
            192,
            kernel_size=[
                3,
            ],
            stride=1,
            padding=1,
        )
        self.pool3 = ivy.MaxPool2D(3, 2, 0)

        self.inception3A = Inception(
            in_channels=192,
            num1x1=64,
            num3x3_reduce=96,
            num3x3=128,
            num5x5_reduce=16,
            num5x5=32,
            pool_proj=32,
        )
        self.inception3B = Inception(
            in_channels=256,
            num1x1=128,
            num3x3_reduce=128,
            num3x3=192,
            num5x5_reduce=32,
            num5x5=96,
            pool_proj=64,
        )
        self.pool4 = ivy.MaxPool2D(3, 2, 0)

        self.inception4A = Inception(
            in_channels=480,
            num1x1=192,
            num3x3_reduce=96,
            num3x3=208,
            num5x5_reduce=16,
            num5x5=48,
            pool_proj=64,
        )
        self.inception4B = Inception(
            in_channels=512,
            num1x1=160,
            num3x3_reduce=112,
            num3x3=224,
            num5x5_reduce=24,
            num5x5=64,
            pool_proj=64,
        )
        self.inception4C = Inception(
            in_channels=512,
            num1x1=128,
            num3x3_reduce=128,
            num3x3=256,
            num5x5_reduce=24,
            num5x5=64,
            pool_proj=64,
        )
        self.inception4D = Inception(
            in_channels=512,
            num1x1=112,
            num3x3_reduce=144,
            num3x3=288,
            num5x5_reduce=32,
            num5x5=64,
            pool_proj=64,
        )
        self.inception4E = Inception(
            in_channels=528,
            num1x1=256,
            num3x3_reduce=160,
            num3x3=320,
            num5x5_reduce=32,
            num5x5=128,
            pool_proj=128,
        )
        self.pool5 = ivy.MaxPool2D(3, 2, 0)

        self.inception5A = Inception(
            in_channels=832,
            num1x1=256,
            num3x3_reduce=160,
            num3x3=320,
            num5x5_reduce=32,
            num5x5=128,
            pool_proj=128,
        )
        self.inception5B = Inception(
            in_channels=832,
            num1x1=384,
            num3x3_reduce=192,
            num3x3=384,
            num5x5_reduce=48,
            num5x5=128,
            pool_proj=128,
        )
        self.pool6 = ivy.AdaptiveAvgPool2d((1, 1))

        self.dropout = ivy.Dropout(0.4)
        self.fc = ivy.Linear(1024, num_classes)

        self.aux4A = Auxiliary(512, num_classes)
        self.aux4D = Auxiliary(528, num_classes)

    def _forward(self, x):
        out = self.conv1(x)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.pool3(out)
        out = self.inception3A(out)
        out = self.inception3B(out)
        out = self.pool4(out)
        out = self.inception4A(out)

        aux1 = self.aux4A(out)

        out = self.inception4B(out)
        out = self.inception4C(out)
        out = self.inception4D(out)

        aux2 = self.aux4D(out)

        out = self.inception4E(out)
        out = self.pool5(out)
        out = self.inception5A(out)
        out = self.inception5B(out)
        out = self.pool6(out)
        out = ivy.flatten(1, out=out)
        out = self.dropout(out)
        out = self.fc(out)

        # we need all 3 because loss func of inceptionNet
        # is weighted avg of these 3 (out, aux1, and aux2)
        return out, aux1, aux2


In [8]:
class ConvBlock(ivy.Module):
    def __init__(self, in_chaivyels, out_chaivyels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()

        # print(f"Sarvesh says in_chaivyels, out_chaivyels and kernel_size is: {in_chaivyels, out_chaivyels, kernel_size}")
        # print(f"kernel_size shape: {ivy.shape(kernel_size)}")
        # shape = (1, 3, 4, 5)
        # kernel_size = np.random.random(shape) 

        if len(ivy.shape(kernel_size)) == 1:
            kernel_size = [kernel_size[0], kernel_size[0]]
        # kernel_size = [1, 1]
        # print(f"Sarvesh says in_chaivyels, out_chaivyels and kernel_size is: {in_chaivyels, out_chaivyels, kernel_size}")

        self.conv = ivy.Conv2D(in_chaivyels, out_chaivyels, kernel_size, stride, padding)
        self.bn = ivy.BatchNorm2D(out_chaivyels)
        self.activation = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x


In [ ]:
class Inception(ivy.Module):
    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        super(Inception, self).__init__()

        # Four output channel for each parallel block of network
        # Note, within Inception the individual blocks are running parallely
        # NOT sequentially.
        self.block1 = ivy.Sequential(
            ConvBlock(
                in_channels,
                num1x1,
                kernel_size=[
                    1,
                ],
                stride=1,
                padding=0,
            )
        )

        self.block2 = ivy.Sequential(
            ConvBlock(
                in_channels,
                num3x3_reduce,
                kernel_size=[
                    1,
                ],
                stride=1,
                padding=0,
            ),
            ConvBlock(
                num3x3_reduce,
                num3x3,
                kernel_size=[
                    3,
                ],
                stride=1,
                padding=1,
            ),
        )

        self.block3 = ivy.Sequential(
            ConvBlock(
                in_channels,
                num5x5_reduce,
                kernel_size=[
                    1,
                ],
                stride=1,
                padding=0,
            ),
            ConvBlock(
                num5x5_reduce,
                num5x5,
                kernel_size=[
                    5,
                ],
                stride=1,
                padding=2,
            ),
        )

        self.block4 = ivy.Sequential(
            ivy.MaxPool2D(3, 1, 1),  # ceil_mode=True
            ConvBlock(
                in_channels,
                pool_proj,
                kernel_size=[
                    1,
                ],
                stride=1,
                padding=0,
            ),
        )

    def _forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(x)
        block3 = self.block3(x)
        block4 = self.block4(x)

        print(f"Sarvesh says shapes of [block1, block2, block3, block4] are: {ivy.shape(block1), ivy.shape(block2), ivy.shape(block3), ivy.shape(block4)}")
        print(f"block1 : \n{ivy.shape(block1)}\n")
        print(f"block2 : \n{ivy.shape(block2)}\n")
        print(f"block3 : \n{ivy.shape(block3)}\n")
        print(f"block4 : \n{ivy.shape(block4)}\n")
        return ivy.concat([block1, block2, block3, block4], axis=3)

In [10]:
class Auxiliary(ivy.Module):
    def __init__(self, in_channels, num_classes):
        super(Auxiliary, self).__init__()

        self.pool = ivy.AdaptiveAvgPool2d((4, 4))
        self.conv = ivy.Conv2D(
            in_channels, 128, [1,1], 1, 0 # understand this convolution operation from scaler  
        ) # no_ofchannels in input image, no_of_kernels, kernel_size, stride, padding
        
        # TODO: covn2d() func input me kya leta hai uska analogy/cvp
        self.activation = ivy.ReLU()

        self.fc1 = ivy.Linear(2048, 1024)
        self.dropout = ivy.Dropout(0.7)
        self.fc2 = ivy.Linear(1024, num_classes)

    def _forward(self, x):
        out = self.pool(x)

        out = self.conv(out)
        out = self.activation(out)
        print("out shape is  ", out.shape)
        # out shape is  torch.Size([2, 128, 4, 4])

        out = ivy.flatten(1, out=out)

        out = self.fc1(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.fc2(out)

        return out


# ipynb kernel helper

In [1]:
pip install -U ipykernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 815.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: debugpy
    Found existing installation: debugpy 1.5.1
    Uninstalling debugpy-1.5.1:
      Successfully uninstalled debugpy-1.5.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.15.0
    Uninstalling ipykernel-6.15.0:
      Successfully uninstalled ipykernel-6.15.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipykernel
ipykernel.__version__

'6.15.0'